In [1]:
!pip install -q langchain-community deep_translator faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [2]:
!gdown 1cDe8rOzd6-FURtn3w7xr8_tRHYIXxHM-
!unzip Database

Downloading...
From: https://drive.google.com/uc?id=1cDe8rOzd6-FURtn3w7xr8_tRHYIXxHM-
To: /content/Database.zip
100% 801k/801k [00:00<00:00, 23.4MB/s]
Archive:  Database.zip
  inflating: fragrance_collection_vnd.csv  
  inflating: inventory.csv           
  inflating: __MACOSX/._inventory.csv  


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# generator = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B-Instruct-AWQ")
# generator = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B-Instruct")
generator = pipeline("text-generation", model="phatvucoder/Qwen2.5-1.5B-Perfumassist")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
import pandas as pd
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

df = pd.read_csv("fragrance_collection_vnd.csv")

def row_to_document(row):
    text = " | ".join([f"{col}: {str(row[col])}" for col in row.index if col not in ['Price', 'image', 'Rate', 'Rating_count', 'Size', 'Year']])
    return Document(page_content=text)

documents = [row_to_document(row) for _, row in df.iterrows()]

documents[:5]

# HuggingFace Embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# FAISS index
db = FAISS.from_documents(documents, embedding_model)

# Save FAISS index
db.save_local("faiss_index")

# Usage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load FAISS index from folder
db = FAISS.load_local("faiss_index",
                      embedding_model,
                      allow_dangerous_deserialization=True)

from deep_translator import GoogleTranslator

translator = GoogleTranslator()

# Similarity search
query = "chai nước hoa nữ"
query = GoogleTranslator(source='vi', target='en').translate(query)
print(query)

results = db.similarity_search(query, k=100)


<ipython-input-4-69e4e1482f8e>:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Women's perfume bottle


In [5]:
import pandas as pd
import sqlite3

# Load CSV files
fragrance_df = pd.read_csv("fragrance_collection_vnd.csv")
inventory_df = pd.read_csv("inventory.csv")

# Tạo connection tới SQLite DB (sẽ tạo file nếu chưa có)
conn = sqlite3.connect("perfumes.db")

# Đưa dữ liệu vào các bảng SQLite
fragrance_df.to_sql("fragrance_collection", conn, if_exists="replace", index=False)
inventory_df.to_sql("inventory", conn, if_exists="replace", index=False)

# Đóng connection
conn.close()

# CLI

In [ ]:
import json
import re
import smtplib
import sqlite3
from email.mime.text import MIMEText
from email.utils import formataddr
from google.colab import userdata
mail_pass_app = userdata.get('mail_pass_app')

# --- PHẦN GỬI EMAIL ---
def send_email(msg):
    # Thông tin tài khoản Gmail (nên lưu dưới biến môi trường)
    gmail_user = 'vhp08072004@gmail.com'
    gmail_password = mail_pass_app  # Mật khẩu ứng dụng

    from_name = 'Siêu chatbot'
    to_email = 'vhphatdz@gmail.com'
    subject = 'YÊU CẦU HỖ TRỢ TỪ KHÁCH HÀNG'
    body = f"Có yêu cầu hỗ trợ từ khách hàng, nội dung: {msg}"

    msg_email = MIMEText(body)
    msg_email['Subject'] = subject
    msg_email['From'] = formataddr((from_name, gmail_user))
    msg_email['To'] = to_email

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(gmail_user, gmail_password)
        server.send_message(msg_email)
        server.quit()
        print('Email đã được gửi thành công!')
    except Exception as e:
        print(f'Lỗi khi gửi email: {e}')

# --- HÀM TRÍCH XUẤT JSON ---
def extract_valid_json(text):
    json_blocks = re.findall(r'\{.*?\}', text, re.DOTALL)
    for block in json_blocks:
        try:
            return json.loads(block)
        except json.JSONDecodeError:
            print(f"JSON không hợp lệ: {block}")
    return None

# --- HÀM GỌI LLM PHÂN TÍCH INPUT ---
def call_llm(user_input):
    prompt = f"""
# ROLE & GOAL:
Bạn là bộ phân tích đầu vào cho chatbot tư vấn nước hoa và chăm sóc khách hàng. Nhiệm vụ của bạn là phân tích **chặt chẽ** `Prompt của người dùng` được cung cấp bên dưới, sau đó trả về **duy nhất** một đối tượng JSON hợp lệ chứa kết quả phân tích theo đúng cấu trúc yêu cầu và cung cấp thông tin định tuyến hữu ích cho Node 2 (Router).

# ANALYSIS DIMENSIONS:
1. **`is_harmful` (Boolean):** Xác định prompt có chứa nội dung độc hại, xúc phạm, lăng mạ, đe dọa, thù ghét, hoặc vi phạm chính sách an toàn hay không.
2. **`is_relevant` (Boolean):** Xác định prompt có liên quan đến nghiệp vụ chính (nước hoa hoặc chăm sóc khách hàng) hay không.
3. **`is_capable` (Boolean):** Xác định liệu chatbot có khả năng xử lý yêu cầu trong prompt hay không. (Chỉ đúng khi: is_harmful=false, is_relevant=true, is_chitchat=false.)
4. **`is_chitchat` (Boolean):** Xác định prompt có phải là giao tiếp xã giao không liên quan trực tiếp đến nghiệp vụ.
5. **`intent` (List[String]):** Xác định ý định chính của người dùng, lựa chọn từ:
    - `ask_info`, `consult`, `compare`, `similar`, `purchase_intent`, `check_stock`, `ask_price`
   (Nếu không rõ hoặc không liên quan, trả về danh sách rỗng [])
6. **`node2_route` (String):** Dựa trên các phân tích, định tuyến:
    - Nếu `is_harmful` true hoặc `is_relevant` false → "rejection_response"
    - Nếu `is_capable` false → "handoff_prompt"
    - Nếu `is_chitchat` true → "simple_response"
    - Nếu không và:
        * Nếu intent chứa "purchase_intent" → "knowledge_retrieval_purchase"
        * Nếu intent chứa "check_stock" hoặc "ask_price" → "direct_tool_use"
        * Nếu intent chứa "ask_info", "consult", "compare", "similar" → "knowledge_retrieval"
    - Nếu không xác định, để là "rejection_response"

# FEW-SHOT EXAMPLES:
## "rejection_response":
Prompt: "shop làm ăn như ***"
Output:
{{
  "is_harmful": true,
  "is_relevant": true,
  "is_capable": false,
  "is_chitchat": false,
  "intent": [],
  "node2_route": "rejection_response"
}}

## "simple_response":
Prompt: "Chào shop"
Output:
{{
  "is_harmful": false,
  "is_relevant": false,
  "is_capable": false,
  "is_chitchat": true,
  "intent": [],
  "node2_route": "simple_response"
}}

## "handoff_prompt":
Prompt: "Mình cần gặp quản lý"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": false,
  "is_chitchat": false,
  "intent": [],
  "node2_route": "handoff_prompt"
}}

## "direct_tool_use":
Prompt: "Chai Dior Sauvage giá bao nhiêu vậy shop?"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["ask_price", "ask_info"],
  "node2_route": "direct_tool_use"
}}

## "knowledge_retrieval":
Prompt: "Tư vấn cho em chai nước hoa nữ đi tiệc mùi ngọt ngào ạ"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["consult", "ask_info"],
  "node2_route": "knowledge_retrieval"
}}

## "knowledge_retrieval_purchase":
Prompt: "Tôi muốn mua nước hoa Chanel, ai giới thiệu giúp tôi"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["purchase_intent", "ask_info"],
  "node2_route": "knowledge_retrieval_purchase"
}}

# USER PROMPT TO ANALYZE:
"{user_input}"

# OUTPUT REQUIREMENTS:
- Chỉ trả về duy nhất một đối tượng JSON hợp lệ mà không có văn bản khác.
- Cấu trúc JSON chính xác như sau:
{{
  "is_harmful": Boolean,
  "is_relevant": Boolean,
  "is_capable": Boolean,
  "is_chitchat": Boolean,
  "intent": [Danh sách chuỗi],
  "node2_route": String (một trong: "rejection_response", "handoff_prompt", "simple_response", "knowledge_retrieval", "direct_tool_use", "knowledge_retrieval_purchase")
}}

# LƯU Ý:
- KHÔNG thêm văn bản, lời giải thích hay định dạng markdown ngoài đối tượng JSON.
- JSON phải không có lỗi cú pháp.
"""
    messages = [{"role": "user", "content": prompt}]
    output = generator(
        messages,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.5,
        top_k=20,
        top_p=0.8
    )[0]["generated_text"][1]["content"]
    print("LLM Output:", output)

    output_json = extract_valid_json(output)
    print("Parsed JSON:", output_json)
    return output_json

# --- HÀM ĐỊNH TUYẾN PHẢN HỒI DỰA TRÊN KẾT QUẢ PHÂN TÍCH ---
def route_response(analysis_result, user_input):
    route = analysis_result.get("node2_route", "")
    if route == "rejection_response":
        print("Xin lỗi bạn, nội dung của bạn không phù hợp hoặc không nằm trong phạm vi hỗ trợ của chúng tôi.")
    elif route == "simple_response":
        messages = [{"role": "user", "content": f'Trả lời một cách dí dỏm câu sau: "{user_input}"'}]
        response = generator(
            messages,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"]
        print(response)
    elif route == "handoff_prompt":
        send_email(user_input)
        print("Yêu cầu của bạn liên quan đến việc trò chuyện trực tiếp với nhân viên hoặc gặp quản lý. Vui lòng chờ, chúng tôi sẽ chuyển tiếp yêu cầu của bạn cho chuyên viên CSKH.")
    elif route == "direct_tool_use":
        print("Đang xử lý yêu cầu kiểm tra giá/tồn kho...")

        # --- Hàm rút trích truy vấn SQL từ output của LLM ---
        def extract_sql_query(text):
            match = re.search(r"```(?:sql)?\s*(.*?)\s*```", text, re.DOTALL | re.IGNORECASE)
            if match:
                return match.group(1).strip()
            if "SELECT" in text.upper():
                return text.strip()
            return None

        # --- Prompt chặt yêu cầu LLM viết truy vấn SQL với few-shot examples ---
        prompt_sql = f"""
Bạn là chuyên gia SQL. Nhiệm vụ của bạn là viết **DUY NHẤT** một câu truy vấn SQL hoàn chỉnh, chỉ chứa **KHỐI TRUY VẤN SQL** duy nhất, để lấy thông tin sản phẩm từ bảng 'fragrance_collection' và thông tin tồn kho từ bảng 'inventory' dựa trên yêu cầu sau: "{user_input}".
Chỉ được trả về truy vấn SQL trong khối code, không có lời giải thích.
Few-shot examples:
Ví dụ 1:
Yêu cầu: "Tìm thông tin sản phẩm tên 'Chanel No.5' và số lượng tồn kho."
Truy vấn:
```
SELECT fc.Name, fc.Price, fc.Description, inv.Quantity
FROM fragrance_collection fc
JOIN inventory inv ON fc.Name = inv.Name
WHERE fc.Name = 'Chanel No.5';
```
Ví dụ 2:
Yêu cầu: "Lấy danh sách các sản phẩm của thương hiệu 'Dior' và số lượng tồn kho của chúng."
Truy vấn:
```
SELECT fc.Name, fc.Brand, fc.Price, inv.Quantity
FROM fragrance_collection fc
JOIN inventory inv ON fc.Name = inv.Name
WHERE fc.Brand = 'Dior';
```
Viết truy vấn SQL cho yêu cầu của người dùng.
"""

        attempts = 0
        sql_query = None
        rows = None
        while attempts < 3 and (sql_query is None or rows is None):
            attempts += 1
            print(f"\nThử tạo và thực hiện truy vấn lần {attempts}...")
            messages = [{"role": "user", "content": prompt_sql}]
            generated_output = generator(
                messages,
                max_new_tokens=150,
                do_sample=True,
                temperature=0.7,
                top_k=20,
                top_p=0.8
            )[0]["generated_text"][1]["content"].strip()
            print("LLM Output:\n", generated_output)
            sql_query = extract_sql_query(generated_output)
            if sql_query is None:
                print("Không rút trích được truy vấn SQL, thử lại...")
                continue
            print("Truy vấn SQL được rút trích:\n", sql_query)
            try:
                conn = sqlite3.connect("perfumes.db")
                cursor = conn.cursor()
                cursor.execute(sql_query)
                rows = cursor.fetchall()
                conn.close()
                if not rows:
                    print("Truy vấn trả về kết quả rỗng, thử lại...")
                    rows = None
            except Exception as e:
                print("Lỗi khi thực hiện truy vấn:", e)
                rows = None

        if sql_query is None or rows is None:
            print("Không thể tạo và thực hiện truy vấn thành công sau 3 lần thử.")
        else:
            context = "\n".join([str(row) for row in rows])
            print("Kết quả truy vấn từ database:")
            print(context)

            final_prompt = f"Dựa trên thông tin sau đây:\n{context}\nHãy tổng hợp và đưa ra câu trả lời chi tiết cho yêu cầu: '{user_input}'"
            messages = [{"role": "user", "content": final_prompt}]
            final_answer = generator(
                messages,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=20,
                top_p=0.8
            )[0]["generated_text"][1]["content"].strip()
            print("Câu trả lời cuối cùng:")
            print(final_answer)
    elif route in ["knowledge_retrieval", "knowledge_retrieval_purchase"]:
        print("Đang truy xuất thông tin từ cơ sở dữ liệu...")

        # --- Bước 1: Hỏi LLM tạo câu truy vấn dựa trên input của người dùng ---
        query_prompt = f"Bạn hãy đưa ra một câu truy vấn ngắn gọn, rõ ràng và tự nhiên nhằm tìm kiếm thông tin liên quan đến yêu cầu: '{user_input}'"
        messages = [{"role": "user", "content": query_prompt}]
        generated_query = generator(
            messages,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"].strip()
        print("Truy vấn do LLM tạo ra:", generated_query)

        # --- Bước 2: Dịch câu truy vấn (nếu cần) và thực hiện truy vấn trên FAISS ---
        generated_query_en = GoogleTranslator(source='vi', target='en').translate(generated_query)
        print("Truy vấn sau khi dịch sang tiếng Anh:", generated_query_en)

        # Sử dụng đối tượng FAISS được load sẵn (đảm bảo biến này không trùng với SQLDatabase)
        results = faiss_db.similarity_search(generated_query_en, k=5)
        context = "\n".join([doc.page_content for doc in results])
        print("Kết quả truy vấn tương đồng:")
        print(context)

        # --- Bước 3: Hỏi LLM tổng hợp câu trả lời cuối cùng dựa trên context thu thập ---
        final_prompt = f"Dựa trên thông tin sau đây:\n{context}\nHãy đưa ra câu trả lời chi tiết và tự nhiên cho câu hỏi của người dùng: '{user_input}'"
        messages = [{"role": "user", "content": final_prompt}]
        final_answer = generator(
            messages,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"].strip()
        print("Câu trả lời cuối cùng:")
        print(final_answer)
    else:
        print("Xin lỗi, có lỗi trong quá trình định tuyến yêu cầu.")

# --- HÀM PHÂN TÍCH INPUT VÀ GỌI ĐỊNH TUYẾN ---
def analyze_user_input(user_input, max_retry=3):
    for i in range(max_retry):
        analysis_result = call_llm(user_input)
        if analysis_result:
            break
        else:
            print("LLM không trả về kết quả hợp lệ.")
            print(f"Đang thử lại lần {i+1}/{max_retry}...")
    else:
        print("LLM không thể xử lý yêu cầu.")
        return None

    route_response(analysis_result, user_input)

# --- LOAD FAISS INDEX (đảm bảo đây là đối tượng FAISS chứ không phải SQLDatabase) ---
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
faiss_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

# --- VÒNG LẶP CHÍNH ---
if __name__ == "__main__":
    print("Nhập 'exit' để kết thúc trò chuyện.")
    while True:
        user_question = input("       You: ").strip()
        if user_question.lower() == "exit":
            print("Tư vấn mỏi mồm luôn r đấy, nhớ mua nước moa nha bro <3")
            break

        print("ChatGPFake:")
        analyze_user_input(user_question)
        print()

Nhập 'exit' để kết thúc trò chuyện.
       You: chào bạn
ChatGPFake:
LLM Output: ```json
{
  "is_harmful": false,
  "is_relevant": false,
  "is_capable": false,
  "is_chitchat": true,
  "intent": [],
  "node2_route": "simple_response"
}
```
Parsed JSON: {'is_harmful': False, 'is_relevant': False, 'is_capable': False, 'is_chitchat': True, 'intent': [], 'node2_route': 'simple_response'}
Chào bạn! Mình chào bạn thế này nhé.



KeyboardInterrupt: Interrupted by user

# UI

In [54]:
!pip install -q -U chainlit streamlit
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [94]:
%%writefile app.py
import chainlit as cl
import json
import re
import smtplib
import sqlite3
from email.mime.text import MIMEText
from email.utils import formataddr

# --------------------------------------------------------------------------------
# Các import liên quan đến mô hình và cơ sở dữ liệu vector
# --------------------------------------------------------------------------------
from transformers import pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from deep_translator import GoogleTranslator

# --------------------------------------------------------------------------------
# Private password for sending mail
# --------------------------------------------------------------------------------
from google.colab import userdata
mail_pass_app = userdata.get('mail_pass_app')

# --------------------------------------------------------------------------------
# Khởi tạo Pipeline
# --------------------------------------------------------------------------------
generator = pipeline("text-generation", model="phatvucoder/Qwen2.5-1.5B-Perfumassist")

# --------------------------------------------------------------------------------
# Khởi tạo model Embeddings và load FAISS Index
# --------------------------------------------------------------------------------
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
faiss_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

# --------------------------------------------------------------------------------
# Kết nối thử với SQLite database (chỉ để demo)
# --------------------------------------------------------------------------------
conn = sqlite3.connect("perfumes.db")
cursor = conn.cursor()

# Kiểm tra 5 dòng đầu tiên của bảng fragrance_collection
cursor.execute("SELECT * FROM fragrance_collection LIMIT 5")
first_5_fragrance = cursor.fetchall()

# Kiểm tra 5 dòng đầu tiên của bảng inventory
cursor.execute("SELECT * FROM inventory LIMIT 5")
first_5_inventory = cursor.fetchall()

conn.close()

# --------------------------------------------------------------------------------
# Hàm gửi email
# --------------------------------------------------------------------------------
def send_email(msg):
    """
    Hàm gửi email khi cần chuyển tiếp yêu cầu của khách hàng
    đến nhân viên hỗ trợ hoặc quản lý.
    """
    gmail_user = 'vhp08072004@gmail.com'
    gmail_password = mail_pass_app  # Mật khẩu ứng dụng

    from_name = 'Siêu chatbot'
    to_email = 'vhphatdz@gmail.com'
    subject = 'YÊU CẦU HỖ TRỢ TỪ KHÁCH HÀNG'
    body = f"Có yêu cầu hỗ trợ từ khách hàng, nội dung: {msg}"

    msg_email = MIMEText(body)
    msg_email['Subject'] = subject
    msg_email['From'] = formataddr((from_name, gmail_user))
    msg_email['To'] = to_email

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(gmail_user, gmail_password)
        server.send_message(msg_email)
        server.quit()
    except Exception as e:
        print(f'Lỗi khi gửi email: {e}')

# --------------------------------------------------------------------------------
# Hàm tách JSON hợp lệ từ đoạn text
# --------------------------------------------------------------------------------
def extract_valid_json(text):
    """
    Tìm khối JSON đầu tiên hợp lệ trong text.
    """
    json_blocks = re.findall(r'\{.*?\}', text, re.DOTALL)
    for block in json_blocks:
        try:
            return json.loads(block)
        except json.JSONDecodeError:
            print(f"JSON không hợp lệ: {block}")
    return None

# --------------------------------------------------------------------------------
# Hàm gọi LLM để phân tích intent
# --------------------------------------------------------------------------------
def call_llm(user_input):
    """
    Gửi Prompt phân tích sang LLM, nhận về chuỗi JSON duy nhất.
    Từ đó chuyển đổi thành dict để trích xuất các giá trị cần thiết.
    """
    prompt = f"""
# ROLE & GOAL:
Bạn là bộ phân tích đầu vào cho chatbot tư vấn nước hoa và chăm sóc khách hàng. Nhiệm vụ của bạn là phân tích **chặt chẽ** Prompt của người dùng được cung cấp bên dưới, sau đó trả về **duy nhất** một đối tượng JSON hợp lệ chứa kết quả phân tích theo đúng cấu trúc yêu cầu và cung cấp thông tin định tuyến hữu ích cho Node 2 (Router).

# ANALYSIS DIMENSIONS:
1. **is_harmful (Boolean):** Xác định prompt có chứa nội dung độc hại, xúc phạm, lăng mạ, đe dọa, thù ghét, hoặc vi phạm chính sách an toàn hay không.
2. **is_relevant (Boolean):** Xác định prompt có liên quan đến nghiệp vụ chính (nước hoa hoặc chăm sóc khách hàng) hay không.
3. **is_capable (Boolean):** Xác định liệu chatbot có khả năng xử lý yêu cầu trong prompt hay không. (Chỉ đúng khi: is_harmful=false, is_relevant=true, is_chitchat=false.)
4. **is_chitchat (Boolean):** Xác định prompt có phải là giao tiếp xã giao không liên quan trực tiếp đến nghiệp vụ.
5. **intent (List[String]):** Xác định ý định chính của người dùng, lựa chọn từ:
    - ask_info, consult, compare, similar, purchase_intent, check_stock, ask_price
   (Nếu không rõ hoặc không liên quan, trả về danh sách rỗng [])
6. **node2_route (String):** Dựa trên các phân tích, định tuyến:
    - Nếu is_harmful true hoặc is_relevant false → "rejection_response"
    - Nếu is_capable false → "handoff_prompt"
    - Nếu is_chitchat true → "simple_response"
    - Nếu không và:
        * Nếu intent chứa "purchase_intent" → "knowledge_retrieval_purchase"
        * Nếu intent chứa "check_stock" hoặc "ask_price" → "direct_tool_use"
        * Nếu intent chứa "ask_info", "consult", "compare", "similar" → "knowledge_retrieval"
    - Nếu không xác định, để là "rejection_response"

# FEW-SHOT EXAMPLES:
## "rejection_response":
Prompt: "shop làm ăn như ***"
Output:
{{
  "is_harmful": true,
  "is_relevant": true,
  "is_capable": false,
  "is_chitchat": false,
  "intent": [],
  "node2_route": "rejection_response"
}}

## "simple_response":
Prompt: "Chào shop"
Output:
{{
  "is_harmful": false,
  "is_relevant": false,
  "is_capable": false,
  "is_chitchat": true,
  "intent": [],
  "node2_route": "simple_response"
}}

## "handoff_prompt":
Prompt: "Mình cần gặp quản lý"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": false,
  "is_chitchat": false,
  "intent": [],
  "node2_route": "handoff_prompt"
}}

## "direct_tool_use":
Prompt: "Chai Dior Sauvage giá bao nhiêu vậy shop?"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["ask_price", "ask_info"],
  "node2_route": "direct_tool_use"
}}

## "knowledge_retrieval":
Prompt: "Tư vấn cho em chai nước hoa nữ đi tiệc mùi ngọt ngào ạ"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["consult", "ask_info"],
  "node2_route": "knowledge_retrieval"
}}

## "knowledge_retrieval_purchase":
Prompt: "Tôi muốn mua nước hoa Chanel, ai giới thiệu giúp tôi"
Output:
{{
  "is_harmful": false,
  "is_relevant": true,
  "is_capable": true,
  "is_chitchat": false,
  "intent": ["purchase_intent", "ask_info"],
  "node2_route": "knowledge_retrieval_purchase"
}}

# USER PROMPT TO ANALYZE:
"{user_input}"

# OUTPUT REQUIREMENTS:
- Chỉ trả về duy nhất một đối tượng JSON hợp lệ mà không có văn bản khác.
- Cấu trúc JSON chính xác như sau:
{{
  "is_harmful": Boolean,
  "is_relevant": Boolean,
  "is_capable": Boolean,
  "is_chitchat": Boolean,
  "intent": [Danh sách chuỗi],
  "node2_route": String (một trong: "rejection_response", "handoff_prompt", "simple_response", "knowledge_retrieval", "direct_tool_use", "knowledge_retrieval_purchase")
}}

# LƯU Ý:
- KHÔNG thêm văn bản, lời giải thích hay định dạng markdown ngoài đối tượng JSON.
- JSON phải không có lỗi cú pháp.
"""
    messages = [{"role": "user", "content": prompt}]
    # Gọi generator
    output = generator(
        messages,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.5,
        top_k=20,
        top_p=0.8
    )[0]["generated_text"][1]["content"]

    # Thử tách JSON từ output
    output_json = extract_valid_json(output)
    return output_json

# --------------------------------------------------------------------------------
# Hàm phân luồng xử lý dựa trên kết quả phân tích
# --------------------------------------------------------------------------------
async def route_response(analysis_result, user_input):
    """
    Dựa trên phân tích JSON (analysis_result), định tuyến đến luồng xử lý phù hợp.
    """
    route = analysis_result.get("node2_route", "")

    # Gửi tin nhắn dạng async
    if route == "rejection_response":
        await cl.Message(
            content="Xin lỗi bạn, nội dung của bạn không phù hợp hoặc không nằm trong phạm vi hỗ trợ của chúng tôi."
        ).send()

    elif route == "simple_response":
        # Gọi LLM để trả lời giao tiếp xã giao
        prompt_for_smalltalk = f'Trả lời một cách dí dỏm câu sau: "{user_input}"'
        messages = [{"role": "user", "content": prompt_for_smalltalk}]
        response = generator(
            messages,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"]
        await cl.Message(content=response).send()

    elif route == "handoff_prompt":
        # Gửi email cho quản lý/CSKH
        send_email(user_input)
        await cl.Message(
            content="Yêu cầu của bạn liên quan đến việc trò chuyện trực tiếp với nhân viên hoặc gặp quản lý. "
                    "Vui lòng chờ, chúng tôi sẽ chuyển tiếp yêu cầu của bạn cho chuyên viên CSKH."
        ).send()

    elif route == "direct_tool_use":
        await cl.Message(content="Đang xử lý yêu cầu kiểm tra giá/tồn kho...").send()

        # Hàm trích xuất câu lệnh SQL
        def extract_sql_query(text):
            """
            Tìm câu SELECT SQL trong text để thực thi.
            """
            match = re.search(r"(?:SELECT|select).*?(?=;|$)", text, re.DOTALL)
            if match:
                return match.group(0).strip()
            return None

        # Prompt yêu cầu LLM viết câu SQL
        prompt_sql = f"""
Bạn là chuyên gia SQL. Nhiệm vụ của bạn là viết **DUY NHẤT** một câu truy vấn SQL hoàn chỉnh,
chỉ chứa **KHỐI TRUY VẤN SQL** duy nhất, để lấy thông tin sản phẩm từ bảng 'fragrance_collection'
và thông tin tồn kho từ bảng 'inventory' dựa trên yêu cầu sau: "{user_input}".
Chỉ được trả về truy vấn SQL trong khối code, không có lời giải thích.

Ví dụ 1:
Yêu cầu: "Tìm thông tin sản phẩm tên 'Chanel No.5' và số lượng tồn kho."
Truy vấn:
SELECT fc.Name, fc.Price, fc.Description, inv.Quantity
FROM fragrance_collection fc
JOIN inventory inv ON fc.Name = inv.Name
WHERE fc.Name = 'Chanel No.5';

Ví dụ 2:
Yêu cầu: "Lấy danh sách các sản phẩm của thương hiệu 'Dior' và số lượng tồn kho của chúng."
Truy vấn:
SELECT fc.Name, fc.Brand, fc.Price, inv.Quantity
FROM fragrance_collection fc
JOIN inventory inv ON fc.Name = inv.Name
WHERE fc.Brand = 'Dior';

Viết truy vấn SQL cho yêu cầu của người dùng.
"""

        attempts = 0
        sql_query = None
        rows = None

        while attempts < 3 and (sql_query is None or rows is None):
            attempts += 1
            messages = [{"role": "user", "content": prompt_sql}]
            generated_output = generator(
                messages,
                max_new_tokens=150,
                do_sample=True,
                temperature=0.7,
                top_k=20,
                top_p=0.8
            )[0]["generated_text"][1]["content"].strip()

            sql_query = extract_sql_query(generated_output)

            if sql_query is None:
                continue

            try:
                conn = sqlite3.connect("perfumes.db")
                cursor = conn.cursor()
                cursor.execute(sql_query)
                rows = cursor.fetchall()
                conn.close()
                if not rows:
                    rows = None
            except Exception as e:
                rows = None

        if sql_query is None or rows is None:
            await cl.Message(
                content="Không thể tạo và thực hiện truy vấn thành công sau 3 lần thử."
            ).send()
        else:
            # Tổng hợp kết quả truy vấn
            context = "\n".join([str(row) for row in rows])
            final_prompt = f"Dựa trên thông tin sau đây:\n{context}\nHãy tổng hợp và đưa ra câu trả lời chi tiết cho yêu cầu: '{user_input}'"
            messages = [{"role": "user", "content": final_prompt}]
            final_answer = generator(
                messages,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=20,
                top_p=0.8
            )[0]["generated_text"][1]["content"].strip()

            await cl.Message(
                content=f"Kết quả truy vấn:\n{context}\n\nCâu trả lời:\n{final_answer}"
            ).send()

    elif route in ["knowledge_retrieval", "knowledge_retrieval_purchase"]:
        await cl.Message(content="Đang truy xuất thông tin từ cơ sở dữ liệu...").send()

        # Bước 1: Hỏi LLM tạo một câu truy vấn tự nhiên
        query_prompt = f"Bạn hãy đưa ra một câu truy vấn ngắn gọn, rõ ràng và tự nhiên nhằm tìm kiếm thông tin liên quan đến yêu cầu: '{user_input}'"
        messages = [{"role": "user", "content": query_prompt}]
        generated_query = generator(
            messages,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"].strip()

        # Bước 2: Dịch truy vấn sang tiếng Anh để tìm kiếm FAISS
        generated_query_en = GoogleTranslator(source='vi', target='en').translate(generated_query)

        results = faiss_db.similarity_search(generated_query_en, k=5)
        context = "\n".join([doc.page_content for doc in results])

        # Bước 3: Hỏi LLM để tổng hợp câu trả lời từ context
        final_prompt = f"Dựa trên thông tin sau đây:\n{context}\nHãy đưa ra câu trả lời chi tiết và tự nhiên cho câu hỏi của người dùng: '{user_input}'"
        messages = [{"role": "user", "content": final_prompt}]
        final_answer = generator(
            messages,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_k=20,
            top_p=0.8
        )[0]["generated_text"][1]["content"].strip()

        await cl.Message(
            content=(
                f"Câu truy vấn: {generated_query}\n\n"
                f"Kết quả tương đồng:\n{context}\n\n"
                f"Câu trả lời cuối cùng:\n{final_answer}"
            )
        ).send()

    else:
        await cl.Message(
            content="Xin lỗi, có lỗi trong quá trình định tuyến yêu cầu."
        ).send()

# --------------------------------------------------------------------------------
# Hàm triển khai toàn bộ logic
# --------------------------------------------------------------------------------
async def analyze_user_input(user_input, max_retry=3):
    """
    Hàm gọi LLM để phân tích, sau đó định tuyến xử lý thích hợp.
    """
    analysis_result = None
    for _ in range(max_retry):
        analysis_result = call_llm(user_input)
        if analysis_result:
            break

    if not analysis_result:
        await cl.Message(
            content="LLM không thể xử lý yêu cầu hoặc trả về JSON không hợp lệ."
        ).send()
        return

    # Thực hiện route
    await route_response(analysis_result, user_input)

# --------------------------------------------------------------------------------
# Sử dụng Chainlit để tương tác
# --------------------------------------------------------------------------------
@cl.on_chat_start
async def start_chat():
    """
    Hàm này sẽ chạy khi phiên trò chuyện mới bắt đầu.
    """
    welcome_text = (
        "Xin chào! Tôi là chatbot tư vấn nước hoa và hỗ trợ khách hàng. "
        "Bạn muốn tìm hiểu hay hỗ trợ gì hôm nay?"
    )
    await cl.Message(content=welcome_text).send()

@cl.on_message
async def main(message: cl.Message):
    """
    Hàm này sẽ chạy khi người dùng gửi tin nhắn.
    Tham số message là một đối tượng 'Message', chứa trường 'content'.
    """
    user_input = message.content.strip()
    if not user_input:
        await cl.Message(content="Bạn chưa nhập gì cả. Vui lòng thử lại.").send()
        return

    # Gọi hàm xử lý chính
    await analyze_user_input(user_input)


Overwriting app.py


In [95]:
!chainlit run app.py --host 0.0.0.0 --port 5000 &> /content/logs.txt &

In [ ]:
import urllib

print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

!lt --port 5000 --subdomain chatbot-dinhcao


Password/Endpoint IP for localtunnel is: 35.240.153.250
your url is: https://chatbot-dinhcao.loca.lt
